In [3]:
from __future__ import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from numpy import inf
import re

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
!pwd
!cd ..
pwd = !pwd
pwd = pwd[0][:-6]
path = pwd + "1_DATA"

/Users/JackTsai/Google Drive/ADS Project/3_Code


In [12]:
df = pd.read_csv( path + "/2016weather_update.csv", encoding=('utf-8'))

In [15]:
df.head()
#df.shape

Unnamed: 0 Conditions           Date Dew Point Events Gust Speed  \
0           0   Overcast  Jan. 01, 2016   25.0 °F    NaN          -   
1           1   Overcast  Jan. 01, 2016   25.0 °F    NaN          -   
2           2   Overcast  Jan. 01, 2016   26.1 °F    NaN          -   
3           3   Overcast  Jan. 01, 2016   26.1 °F    NaN          -   
4           4   Overcast  Jan. 01, 2016   26.1 °F    NaN   16.1 mph   

  Heat Index Humidity Precip  Pressure    Temp. Time (EST) Visibility  \
0        NaN      51%    NaN  30.06 in  42.1 °F   12:51 AM    10.0 mi   
1        NaN      53%    NaN  30.06 in  41.0 °F    1:51 AM    10.0 mi   
2        NaN      55%    NaN  30.06 in  41.0 °F    2:51 AM    10.0 mi   
3        NaN      55%    NaN  30.06 in  41.0 °F    3:51 AM    10.0 mi   
4        NaN      58%    NaN  30.04 in  39.9 °F    4:51 AM    10.0 mi   

   Wind Dir Wind Speed Windchill  
0  Variable    4.6 mph   39.2 °F  
1  Variable    3.5 mph   39.0 °F  
2      West    4.6 mph   38.0 °F  
3      West    9.2 mph   35.2 °F  
4      West   10.4 mph   33.4 °F

In [16]:
df["Date"] = pd.to_datetime(df["Date"])

In [17]:
#df["Time (EST)"] = df["Time (EST)"])

In [18]:
#df = df[pd.to_datetime(df["Time (EST)"]).dt.minute==51]
#df.head()

In [19]:
#df.shape

In [20]:
df["Temp."].isnull().values.any()

False

In [21]:
df_relevant = df[['Date','Temp.','Humidity','Conditions',"Time (EST)"]].copy()
#df1231.columns = ['Temperature', 'Humidity', 'Conditions']
df_relevant.rename(columns={'Temp.': 'Temperature',"Time (EST)":'Time'}, inplace=True)
df_relevant.head()

Date Temperature Humidity Conditions      Time
0 2016-01-01     42.1 °F      51%   Overcast  12:51 AM
1 2016-01-01     41.0 °F      53%   Overcast   1:51 AM
2 2016-01-01     41.0 °F      55%   Overcast   2:51 AM
3 2016-01-01     41.0 °F      55%   Overcast   3:51 AM
4 2016-01-01     39.9 °F      58%   Overcast   4:51 AM

In [22]:
#dfrelevant_mean['Temperature'] = dfrelevant_mean['Temperature'].str.split('°').str[0]
pattern = re.compile(r'([\d.]*).*')
def extract_beginning_number(a):
    match = pattern.match(a)
    if match:
        return match.group(1)
    else:
        return np.nan
df_relevant['Temperature'] = pd.to_numeric(df_relevant['Temperature'].apply(extract_beginning_number))
df_relevant['Humidity'] = pd.to_numeric(df_relevant['Humidity'].apply(extract_beginning_number))
# dfrelevant_mean['Temperature'] = dfrelevant_mean['Temperature'].str.split('Â').str[0]
#dfrelevant_mean['Temperature'] = dfrelevant_mean["Temperature"].str.strip(" ")
df_relevant.head()

Date  Temperature  Humidity Conditions      Time
0 2016-01-01         42.1      51.0   Overcast  12:51 AM
1 2016-01-01         41.0      53.0   Overcast   1:51 AM
2 2016-01-01         41.0      55.0   Overcast   2:51 AM
3 2016-01-01         41.0      55.0   Overcast   3:51 AM
4 2016-01-01         39.9      58.0   Overcast   4:51 AM

In [55]:
conditions_list = df_relevant.Conditions.unique()
def aggregate_4hour(group):
    dt_total = group.dt.sum()
    conditions = {
        con: np.dot(group.Conditions == con, group.dt) / dt_total
        for con in conditions_list
    }
    return pd.Series(dict(
        Date=group.index.min().date(),
        Temperature=np.dot(group.Temperature, group.dt) / dt_total,
        Humidity=np.dot(group.Humidity, group.dt) / dt_total,
        **conditions
    ))

df_relevant.index = pd.to_datetime((
    df_relevant.Date.dt.strftime('%m-%d-%Y') + " " + pd.to_datetime(df_relevant.Time).dt.strftime('%H:%M:%S')))
df_relevant['dt'] = np.append(np.diff(df_relevant.index)/np.timedelta64(1,'h'), 1.)
df_updated = df_relevant.groupby(pd.TimeGrouper(freq = '4H')).apply(aggregate_4hour)
df_updated['TIMEINT'] = ((df_updated.index.hour/4)+1).astype(int)
df_updated.head(30)


Clear       Date Fog Haze Heavy Rain Heavy Snow  \
2016-01-01 00:00:00         0 2016-01-01   0    0          0          0   
2016-01-01 04:00:00         0 2016-01-01   0    0          0          0   
2016-01-01 08:00:00         0 2016-01-01   0    0          0          0   
2016-01-01 12:00:00         0 2016-01-01   0    0          0          0   
2016-01-01 16:00:00      0.75 2016-01-01   0    0          0          0   
2016-01-01 20:00:00         1 2016-01-01   0    0          0          0   
2016-01-02 00:00:00      0.75 2016-01-02   0    0          0          0   
2016-01-02 04:00:00       0.5 2016-01-02   0    0          0          0   
2016-01-02 08:00:00         0 2016-01-02   0    0          0          0   
2016-01-02 12:00:00       0.5 2016-01-02   0    0          0          0   
2016-01-02 16:00:00      0.75 2016-01-02   0    0          0          0   
2016-01-02 20:00:00         1 2016-01-02   0    0          0          0   
2016-01-03 00:00:00         1 2016-01-03   0    0          0          0   
2016-01-03 04:00:00         1 2016-01-03   0    0          0          0   
2016-01-03 08:00:00         1 2016-01-03   0    0          0          0   
2016-01-03 12:00:00      0.75 2016-01-03   0    0          0          0   
2016-01-03 16:00:00       0.5 2016-01-03   0    0          0          0   
2016-01-03 20:00:00       0.5 2016-01-03   0    0          0          0   
2016-01-04 00:00:00         0 2016-01-04   0    0          0          0   
2016-01-04 04:00:00         0 2016-01-04   0    0          0          0   
2016-01-04 08:00:00       0.5 2016-01-04   0    0          0          0   
2016-01-04 12:00:00         1 2016-01-04   0    0          0          0   
2016-01-04 16:00:00      0.75 2016-01-04   0    0          0          0   
2016-01-04 20:00:00  0.833333 2016-01-04   0    0          0          0   
2016-01-05 00:00:00         1 2016-01-05   0    0          0          0   
2016-01-05 04:00:00         1 2016-01-05   0    0          0          0   
2016-01-05 08:00:00         1 2016-01-05   0    0          0          0   
2016-01-05 12:00:00         1 2016-01-05   0    0          0          0   
2016-01-05 16:00:00         1 2016-01-05   0    0          0          0   
2016-01-05 20:00:00         1 2016-01-05   0    0          0          0   

                    Humidity Light Freezing Fog Light Freezing Rain  \
2016-01-01 00:00:00     53.5                  0                   0   
2016-01-01 04:00:00    58.25                  0                   0   
2016-01-01 08:00:00     53.5                  0                   0   
2016-01-01 12:00:00    49.25                  0                   0   
2016-01-01 16:00:00     48.5                  0                   0   
2016-01-01 20:00:00     50.5                  0                   0   
2016-01-02 00:00:00     48.5                  0                   0   
2016-01-02 04:00:00     53.5                  0                   0   
2016-01-02 08:00:00       47                  0                   0   
2016-01-02 12:00:00     43.5                  0                   0   
2016-01-02 16:00:00    44.75                  0                   0   
2016-01-02 20:00:00     50.5                  0                   0   
2016-01-03 00:00:00     52.5                  0                   0   
2016-01-03 04:00:00     53.5                  0                   0   
2016-01-03 08:00:00    44.25                  0                   0   
2016-01-03 12:00:00       41                  0                   0   
2016-01-03 16:00:00    46.75                  0                   0   
2016-01-03 20:00:00    55.75                  0                   0   
2016-01-04 00:00:00       55                  0                   0   
2016-01-04 04:00:00     50.5                  0                   0   
2016-01-04 08:00:00    43.75                  0                   0   
2016-01-04 12:00:00       33                  0                   0   
2016-01-04 16:00:00    45.25                  0                  

In [56]:
df_updated[df_updated.Date == "2016-01-01"]

Clear       Date Fog Haze Heavy Rain Heavy Snow Humidity  \
2016-01-01 00:00:00     0 2016-01-01   0    0          0          0     53.5   
2016-01-01 04:00:00     0 2016-01-01   0    0          0          0    58.25   
2016-01-01 08:00:00     0 2016-01-01   0    0          0          0     53.5   
2016-01-01 12:00:00     0 2016-01-01   0    0          0          0    49.25   
2016-01-01 16:00:00  0.75 2016-01-01   0    0          0          0     48.5   
2016-01-01 20:00:00     1 2016-01-01   0    0          0          0     50.5   

                    Light Freezing Fog Light Freezing Rain Light Rain  \
2016-01-01 00:00:00                  0                   0          0   
2016-01-01 04:00:00                  0                   0          0   
2016-01-01 08:00:00                  0                   0          0   
2016-01-01 12:00:00                  0                   0          0   
2016-01-01 16:00:00                  0                   0          0   
2016-01-01 20:00:00                  0                   0          0   

                    Light Snow Mostly Cloudy Overcast Partly Cloudy Rain  \
2016-01-01 00:00:00          0             0        1             0    0   
2016-01-01 04:00:00          0             0        1             0    0   
2016-01-01 08:00:00          0             0     0.75          0.25    0   
2016-01-01 12:00:00          0             0        1             0    0   
2016-01-01 16:00:00          0             0        0          0.25    0   
2016-01-01 20:00:00          0             0        0             0    0   

                    Scattered Clouds Snow Temperature Unknown  TIMEINT  
2016-01-01 00:00:00                0    0      41.275       0        1  
2016-01-01 04:00:00                0    0       39.45       0        2  
2016-01-01 08:00:00                0    0      39.675       0        3  
2016-01-01 12:00:00                0    0       39.45       0        4  
2016-01-01 16:00:00                0    0       37.45       0        5  
2016-01-01 20:00:00                0    0      35.325       0        6

In [24]:
df_updated.to_csv('weather_combined.csv',encoding='utf-8')


In [66]:
#df_weather = pd.read_csv(path+"/weather_combined.csv")
#df_weather["Date"] = pd.to_datetime(df["Date"])
df_weather = df_updated.reset_index(drop=True).drop(['Unknown'], axis=1)
df_weather.head()

Clear       Date Fog Haze Heavy Rain Heavy Snow Humidity Light Freezing Fog  \
0     0 2016-01-01   0    0          0          0     53.5                  0   
1     0 2016-01-01   0    0          0          0    58.25                  0   
2     0 2016-01-01   0    0          0          0     53.5                  0   
3     0 2016-01-01   0    0          0          0    49.25                  0   
4  0.75 2016-01-01   0    0          0          0     48.5                  0   

  Light Freezing Rain Light Rain Light Snow Mostly Cloudy Overcast  \
0                   0          0          0             0        1   
1                   0          0          0             0        1   
2                   0          0          0             0     0.75   
3                   0          0          0             0        1   
4                   0          0          0             0        0   

  Partly Cloudy Rain Scattered Clouds Snow Temperature  TIMEINT  
0             0    0                0    0      41.275        1  
1             0    0                0    0       39.45        2  
2          0.25    0                0    0      39.675        3  
3             0    0                0    0       39.45        4  
4          0.25    0                0    0       37.45        5

In [67]:
df_weather[df_weather.Date == "2016-01-01"]

Clear       Date Fog Haze Heavy Rain Heavy Snow Humidity Light Freezing Fog  \
0     0 2016-01-01   0    0          0          0     53.5                  0   
1     0 2016-01-01   0    0          0          0    58.25                  0   
2     0 2016-01-01   0    0          0          0     53.5                  0   
3     0 2016-01-01   0    0          0          0    49.25                  0   
4  0.75 2016-01-01   0    0          0          0     48.5                  0   
5     1 2016-01-01   0    0          0          0     50.5                  0   

  Light Freezing Rain Light Rain Light Snow Mostly Cloudy Overcast  \
0                   0          0          0             0        1   
1                   0          0          0             0        1   
2                   0          0          0             0     0.75   
3                   0          0          0             0        1   
4                   0          0          0             0        0   
5                   0          0          0             0        0   

  Partly Cloudy Rain Scattered Clouds Snow Temperature  TIMEINT  
0             0    0                0    0      41.275        1  
1             0    0                0    0       39.45        2  
2          0.25    0                0    0      39.675        3  
3             0    0                0    0       39.45        4  
4          0.25    0                0    0       37.45        5  
5             0    0                0    0      35.325        6

In [68]:
df_mta = pd.read_csv(path+"/mta_combined.csv")
df_mta["Date"] = pd.to_datetime(df_mta["DATE"])
df_mta = df_mta[["STATION","Date","TIMEINT","ENTRIES"]].copy()
df_mta.head()


STATION       Date  TIMEINT     ENTRIES
0    1 AV 2016-01-01        1  1407774726
1    1 AV 2016-01-01        2  1407775796
2    1 AV 2016-01-01        3  1407776768
3    1 AV 2016-01-01        4  1407779799
4    1 AV 2016-01-01        5  1407783611

In [69]:
df_mta.head(5)

STATION       Date  TIMEINT     ENTRIES
0    1 AV 2016-01-01        1  1407774726
1    1 AV 2016-01-01        2  1407775796
2    1 AV 2016-01-01        3  1407776768
3    1 AV 2016-01-01        4  1407779799
4    1 AV 2016-01-01        5  1407783611

In [70]:
df_weather.head()

Clear       Date Fog Haze Heavy Rain Heavy Snow Humidity Light Freezing Fog  \
0     0 2016-01-01   0    0          0          0     53.5                  0   
1     0 2016-01-01   0    0          0          0    58.25                  0   
2     0 2016-01-01   0    0          0          0     53.5                  0   
3     0 2016-01-01   0    0          0          0    49.25                  0   
4  0.75 2016-01-01   0    0          0          0     48.5                  0   

  Light Freezing Rain Light Rain Light Snow Mostly Cloudy Overcast  \
0                   0          0          0             0        1   
1                   0          0          0             0        1   
2                   0          0          0             0     0.75   
3                   0          0          0             0        1   
4                   0          0          0             0        0   

  Partly Cloudy Rain Scattered Clouds Snow Temperature  TIMEINT  
0             0    0                0    0      41.275        1  
1             0    0                0    0       39.45        2  
2          0.25    0                0    0      39.675        3  
3             0    0                0    0       39.45        4  
4          0.25    0                0    0       37.45        5

In [71]:
df_final= pd.merge(df_mta, df_weather, on = ["Date","TIMEINT"], how = 'left')
df_final

STATION       Date  TIMEINT     ENTRIES     Clear  Fog Haze  \
0            1 AV 2016-01-01        1  1407774726         0    0    0   
1            1 AV 2016-01-01        2  1407775796         0    0    0   
2            1 AV 2016-01-01        3  1407776768         0    0    0   
3            1 AV 2016-01-01        4  1407779799         0    0    0   
4            1 AV 2016-01-01        5  1407783611      0.75    0    0   
5            1 AV 2016-01-01        6  1407786852         1    0    0   
6            1 AV 2016-01-02        1  1407788395      0.75    0    0   
7            1 AV 2016-01-02        2  1407788714       0.5    0    0   
8            1 AV 2016-01-02        3  1407790321         0    0    0   
9            1 AV 2016-01-02        4  1407794225       0.5    0    0   
10           1 AV 2016-01-02        5  1407798688      0.75    0    0   
11           1 AV 2016-01-02        6  1407802820         1    0    0   
12           1 AV 2016-01-03        1  1407805211         1    0    0   
13           1 AV 2016-01-03        2  1407805739         1    0    0   
14           1 AV 2016-01-03        3  1407807196         1    0    0   
15           1 AV 2016-01-03        4  1407810515      0.75    0    0   
16           1 AV 2016-01-03        5  1407814293       0.5    0    0   
17           1 AV 2016-01-03        6  1407817257       0.5    0    0   
18           1 AV 2016-01-04        1  1407818153         0    0    0   
19           1 AV 2016-01-04        2  1407818810         0    0    0   
20           1 AV 2016-01-04        3  1407824411       0.5    0    0   
21           1 AV 2016-01-04        4  1407828020         1    0    0   
22           1 AV 2016-01-04        5  1407835582      0.75    0    0   
23           1 AV 2016-01-04        6  1407839057  0.833333    0    0   
24           1 AV 2016-01-05        1  1407840037         1    0    0   
25           1 AV 2016-01-05        2  1407840684         1    0    0   
26           1 AV 2016-01-05        3  1407846891         1    0    0   
27           1 AV 2016-01-05        4  1407850364         1    0    0   
28           1 AV 2016-01-05        5  1407857897         1    0    0   
29           1 AV 2016-01-05        6  1407861698         1    0    0   
...           ...        ...      ...         ...       ...  ...  ...   
765584  ZEREGA AV 2016-12-28        1    54254719       0.5    0    0   
765585  ZEREGA AV 2016-12-28        2    54255305      0.25    0    0   
765586  ZEREGA AV 2016-12-28        3    54255887         0    0    0   
765587  ZEREGA AV 2016-12-28        4    54256342         0    0    0   
765588  ZEREGA AV 2016-12-28        5    54256827       0.5    0    0   
765589  ZEREGA AV 2015-12-29        1    53464730       NaN  NaN  NaN   
765590  ZEREGA AV 2015-12-29        2    53465300       NaN  NaN  NaN   
765591  ZEREGA AV 2015-12-29        3    53465810       NaN  NaN  NaN   
765592  ZEREGA AV 2015-12-29        4    53466238       NaN  NaN  NaN   
765593  ZEREGA AV 2015-12-29        5    53466605       NaN  NaN  NaN   
765594  ZEREGA AV 2016-12-29        1    54256974         1    0    0   
765595  ZEREGA AV 2016-12-29        2    54257536      0.25    0    0   
765596  ZEREGA AV 2016-12-29        3    54258116         0    0    0   
765597  ZEREGA AV 2016-12-29        4    54258578         0    0    0   
765598  ZEREGA AV 2016-12-29        5    54259006       0.5    0    0   
765599  ZEREGA AV 2015-12-30        1    53466728       NaN  NaN  NaN   
765600  ZEREGA AV 2015-12-30        2    53467306       NaN  NaN  NaN   
765601  ZEREGA AV 2015-12-30        3    53467857       NaN  NaN  NaN   
765602  ZEREGA AV 2015-12-30        4    53468326       NaN  NaN  NaN   
765603  ZEREGA AV 2015-12-30        5    53468755       NaN  NaN  NaN   
765604  ZEREGA AV 2016-12-30        1    54259140         1    0    0   
765605  ZEREGA AV 2016-12-30        2    54259629         1    0    0   
765606  ZEREGA AV 2016-12-30        3    54260220       0.5    0    

In [72]:
df_final[(df_final.STATION == '1 AV') & (df_final.Date == '2016-01-01')]

STATION       Date  TIMEINT     ENTRIES Clear Fog Haze Heavy Rain  \
0    1 AV 2016-01-01        1  1407774726     0   0    0          0   
1    1 AV 2016-01-01        2  1407775796     0   0    0          0   
2    1 AV 2016-01-01        3  1407776768     0   0    0          0   
3    1 AV 2016-01-01        4  1407779799     0   0    0          0   
4    1 AV 2016-01-01        5  1407783611  0.75   0    0          0   
5    1 AV 2016-01-01        6  1407786852     1   0    0          0   

  Heavy Snow Humidity     ...     Light Freezing Rain Light Rain Light Snow  \
0          0     53.5     ...                       0          0          0   
1          0    58.25     ...                       0          0          0   
2          0     53.5     ...                       0          0          0   
3          0    49.25     ...                       0          0          0   
4          0     48.5     ...                       0          0          0   
5          0     50.5     ...                       0          0          0   

  Mostly Cloudy Overcast Partly Cloudy Rain Scattered Clouds Snow Temperature  
0             0        1             0    0                0    0      41.275  
1             0        1             0    0                0    0       39.45  
2             0     0.75          0.25    0                0    0      39.675  
3             0        1             0    0                0    0       39.45  
4             0        0          0.25    0                0    0       37.45  
5             0        0             0    0                0    0      35.325  

[6 rows x 21 columns]

In [46]:
df_final.groupby(['STATION', 'Date']).count()['ENTRIES']

STATION    Date      
1 AV       2016-01-01    24
           2016-01-02    24
           2016-01-03    24
           2016-01-04    22
           2016-01-05    22
           2016-01-06    24
           2016-01-07    28
           2016-01-08    31
           2016-01-09    32
           2016-01-10    46
           2016-01-11    24
           2016-01-12    24
           2016-01-13    24
           2016-01-14    24
           2016-01-15    24
           2016-01-16    35
           2016-01-17    34
           2016-01-18    26
           2016-01-19    24
           2016-01-20    24
           2016-01-21    24
           2016-01-22    27
           2016-01-23    45
           2016-01-24    29
           2016-01-25    24
           2016-01-26    24
           2016-01-27    24
           2016-01-28    24
           2016-01-29    24
           2016-01-30    24
                         ..
ZEREGA AV  2016-02-16    50
           2016-02-17    22
           2016-02-18    20
           2016-02-19    2

In [75]:
df_final.to_csv(path+'/final_dataset.csv',encoding='utf-8', index=False)

In [0]:
df_f = pd.read_csv('final_dataset.csv')
df_f.head()

Unnamed: 0 STATION        Date  TIMEINT     ENTRIES  Temperature  Humidity  \
0           0    1 AV  2016-01-01        1  1407774726       41.275      53.5   
1           1    1 AV  2016-01-01        1  1407774726       34.000      48.5   
2           2    1 AV  2016-01-01        1  1407774726       35.775      52.5   
3           3    1 AV  2016-01-01        1  1407774726       35.275      55.0   
4           4  103 ST  2016-01-01        1   124276833       41.275      53.5   

   Heavy Rain  Heavy Snow  Light Freezing Fog  ...    Light Snow  \
0         0.0         0.0                 0.0  ...           0.0   
1         0.0         0.0                 0.0  ...           0.0   
2         0.0         0.0                 0.0  ...           0.0   
3         0.0         0.0                 0.0  ...           0.0   
4         0.0         0.0                 0.0  ...           0.0   

   Mostly Cloudy  Overcast  Partly Cloudy  Rain  Scattered Clouds  Snow  Haze  \
0           0.00      1.00           0.00   0.0               0.0   0.0   0.0   
1           0.25      0.00           0.00   0.0               0.0   0.0   0.0   
2           0.00      0.00           0.00   0.0               0.0   0.0   0.0   
3           0.00      0.75           0.25   0.0               0.0   0.0   0.0   
4           0.00      1.00           0.00   0.0               0.0   0.0   0.0   

   Fog  Clear  
0  0.0   0.00  
1  0.0   0.75  
2  0.0   1.00  
3  0.0   0.00  
4  0.0   0.00  

[5 rows x 22 columns]

In [0]:
df_f.drop(["Unnamed: 0"],axis =1)

STATION        Date  TIMEINT     ENTRIES  Temperature  \
0                  1 AV  2016-01-01        1  1407774726       41.275   
1                  1 AV  2016-01-01        1  1407774726       34.000   
2                  1 AV  2016-01-01        1  1407774726       35.775   
3                  1 AV  2016-01-01        1  1407774726       35.275   
4                103 ST  2016-01-01        1   124276833       41.275   
5                103 ST  2016-01-01        1   124276833       34.000   
6                103 ST  2016-01-01        1   124276833       35.775   
7                103 ST  2016-01-01        1   124276833       35.275   
8         103 ST-CORONA  2016-01-01        1    61101554       41.275   
9         103 ST-CORONA  2016-01-01        1    61101554       34.000   
10        103 ST-CORONA  2016-01-01        1    61101554       35.775   
11        103 ST-CORONA  2016-01-01        1    61101554       35.275   
12               104 ST  2016-01-01        1  3861195697       41.275   
13               104 ST  2016-01-01        1  3861195697       34.000   
14               104 ST  2016-01-01        1  3861195697       35.775   
15               104 ST  2016-01-01        1  3861195697       35.275   
16               110 ST  2016-01-01        1    25164897       41.275   
17               110 ST  2016-01-01        1    25164897       34.000   
18               110 ST  2016-01-01        1    25164897       35.775   
19               110 ST  2016-01-01        1    25164897       35.275   
20               111 ST  2016-01-01        1   110712424       41.275   
21               111 ST  2016-01-01        1   110712424       34.000   
22               111 ST  2016-01-01        1   110712424       35.775   
23               111 ST  2016-01-01        1   110712424       35.275   
24               116 ST  2016-01-01        1  1709330711       41.275   
25               116 ST  2016-01-01        1  1709330711       34.000   
26               116 ST  2016-01-01        1  1709330711       35.775   
27               116 ST  2016-01-01        1  1709330711       35.275   
28      116 ST-COLUMBIA  2016-01-01        1  1209304607       41.275   
29      116 ST-COLUMBIA  2016-01-01        1  1209304607       34.000   
...                 ...         ...      ...         ...          ...   
762035   VAN SICLEN AVE  2016-03-16        6    13396304       34.825   
762036   VAN SICLEN AVE  2016-03-16        6    13396304       38.400   
762037   VAN SICLEN AVE  2016-03-16        6    13396304       34.325   
762038   W 4 ST-WASH SQ  2016-03-16        6   804654480       34.825   
762039   W 4 ST-WASH SQ  2016-03-16        6   804654480       38.400   
762040   W 4 ST-WASH SQ  2016-03-16        6   804654480       34.325   
762041    WAKEFIELD/241  2016-03-16        6    16348575       34.825   
762042    WAKEFIELD/241  2016-03-16        6    16348575       38.400   
762043    WAKEFIELD/241  2016-03-16        6    16348575       34.325   
762044          WALL ST  2016-03-16        6   534194483       34.825   
762045          WALL ST  2016-03-16        6   534194483       38.400   
762046          WALL ST  2016-03-16        6   534194483       34.325   
762047    WEST FARMS SQ  2016-03-16        6   438530467       34.825   
762048    WEST FARMS SQ  2016-03-16        6   438530467       38.400   
762049    WEST FARMS SQ  2016-03-16        6   438530467       34.325   
762050  WHITEHALL S-FRY  2016-03-16        6   266199476       34.825   
762051  WHITEHALL S-FRY  2016-03-16        6   266199476       38.400   
762052  WHITEHALL S-FRY  2016-03-16        6   266199476       34.325   
762053      WINTHROP ST  2016-03-16        6    31797792       34.825   
762054      WINTHROP ST  2016-03-16        6    31797792       38.400   
762055      WINTHROP ST  2016-03-16        6    31797792       34.325   
762056   WOODHAVEN BLVD  2016-03-16        6    56147736       34.825   
762057   WOODHAVEN BLVD  2016-03-16        6    56147736       38.400   
7

In [0]:
# df_relevant.Conditions.unique()

In [0]:
#dfrelevant_dummy = pd.get_dummies(df_relevant.Conditions,prefix='',prefix_sep='')
#dfrelevant_dummy.head()

In [0]:
#dfrelevant_dummy = dfrelevant_dummy.groupby(np.arange(len(dfrelevant_dummy))//4).sum()

In [0]:
#dfrelevant_dummy.head()

In [0]:
#print(dfrelevant_dummy.shape)
#df_relevant_4throw = df_relevant[df_relevant.index % 4 == 0] 
#df_relevant_4throw.head()

In [0]:
#df_relevant_4throw= df_relevant_4throw.reset_index()

In [0]:
#df_relevant_date = df_relevant_4throw.drop(['Temperature','Humidity','Conditions'], axis = 1)
#df_relevant_date.head()


In [0]:
# df_relevant_date.shape

In [0]:
# dfrelevant_mean = df_relevant.drop(['Conditions','Date'], axis = 1)
# dfrelevant_mean.head()

In [0]:
# np.diff(df_relevant.time)

In [0]:
#dfrelevant_mean['Temperature'] = pd.Series(dfrelevant_mean['Temperature']).convert_objects(convert_numeric=True)
#dfrelevant_mean['Humidity'] = pd.Series(dfrelevant_mean['Humidity']).convert_objects(convert_numeric=True)

In [0]:
# dfrelevant_mean['Temperature']= pd.to_numeric(dfrelevant_mean['Temperature'], errors='coerce')
# dfrelevant_mean['Humidity']= pd.to_numeric(dfrelevant_mean['Humidity'], errors='coerce')

In [0]:
# dfrelevant_mean

In [0]:
# dfrelevant_mean.shape

In [0]:
# dfrelevant_mean = dfrelevant_mean.groupby(np.arange(len(dfrelevant_mean))//4).mean()

In [0]:
# dfrelevant_mean.head()